In [2]:
import cv2
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm


### Load and slice videos for training

In [3]:
dataset_path = 'PushUps'
filenames = glob(dataset_path + '\*.avi')

train = pd.DataFrame()
train['video_name'] = filenames
train.shape

(1, 1)

In [4]:
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])

train['tag'] = train_video_tag
train.shape

(1, 2)

In [5]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 2 == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('\\')[1] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.53it/s]


### Load OpenPose COCO model

In [6]:
protoFile = 'Model/pose_deploy_linevec.prototxt'
weightsFile = 'Model/pose_iter_440000.caffemodel'

In [7]:
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [8]:
dataset_frame_path = 'train_1'
frame_filenames = glob(dataset_frame_path + '\*.jpg')

trainFrame = pd.DataFrame()
trainFrame['frames'] = frame_filenames
trainFrame.shape

(28, 1)

In [9]:
inWidth = 368
inHeight = 368

imgPaths = []
for frame in trainFrame['frames']:
    img = cv2.imread(frame)
#     img = cv2.resize(img, (inWidth, inHeight))
    imgPaths.append(img)
    
# cv2.imshow('img', imgPaths[0])
# cv2.waitKey(3)

In [10]:
inBlob = cv2.dnn.blobFromImages(imgPaths, 1.0/255, (imgPaths[0].shape[1], imgPaths[0].shape[0]), (0,0,0), swapRB=False, crop=False)
net.setInput(inBlob)
output = net.forward()

In [20]:
output.shape

(28, 57, 30, 40)

In [14]:
frameHeight, frameWidth, _ = imgPaths[0].shape
print(frameWidth)

320


In [27]:
H = output.shape[2]
W = output.shape[3]
points = []

for i in range(output.shape[1]):
    probMap = output[0,i,:,:]
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
    x = (frameHeight * point[0]) / W
    y = (frameWidth * point[1]) / H
    
    if prob > 0.9:
        cv2.circle(imgPaths[0], (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(imgPaths[0], "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)  
        points.append((int(x), int(y)))
    else:
        points.append(None)
cv2.imshow("Out", imgPaths[0])
cv2.waitKey(0)
cv2.destroyAllWindows()    